In [11]:
pip install PyMuPDF pdfplumber Pillow


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import fitz  # PyMuPDF
import os
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import os
import fitz  # PyMuPDF
from PIL import Image
import pytesseract
from transformers import CLIPProcessor, CLIPModel

# Load CLIP Model Once
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def is_graph_chart(image_path):
    """Classify image as graph/chart or not."""
    image = Image.open(image_path)
    inputs = processor(text=["a chart", "a graph", "a bar chart", "a random image"], images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    labels = ["chart", "graph", "bar chart", "random image"]
    best_label = labels[probs.argmax()]
    return best_label in ["chart", "graph", "bar chart"]

def extract_text_from_image(image_path):
    """Perform OCR on image and return extracted text."""
    image = Image.open(image_path).convert('L')
    text = pytesseract.image_to_string(image)
    return text.strip()

def save_full_page_as_image(page, page_num, output_dir):
    """Render the full page as an image."""
    zoom_x = 2.0  # 2x Resolution
    zoom_y = 2.0
    mat = fitz.Matrix(zoom_x, zoom_y)
    pix = page.get_pixmap(matrix=mat)
    os.makedirs(output_dir, exist_ok=True)
    image_path = f"{output_dir}/page_{page_num + 1}.png"
    pix.save(image_path)
    return image_path

def extract_pdf_text_and_images(pdf_path, output_image_dir="extracted_images", filtered_dir="filtered_images", extracted_text_dir="extracted_text"):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(filtered_dir, exist_ok=True)
    os.makedirs(extracted_text_dir, exist_ok=True)

    doc = fitz.open(pdf_path)
    all_text = ""
    meaningful_count = 0

    for page_num, page in enumerate(doc):
        page_text = page.get_text()
        all_text += f"\n--- Page {page_num + 1} ---\n{page_text}\n"

        # Render full page as image
        full_page_image_path = save_full_page_as_image(page, page_num, output_image_dir)

        # Check if this page contains a graph/chart
        if is_graph_chart(full_page_image_path):
            meaningful_filename = f"{filtered_dir}/page_{page_num + 1}.png"
            os.rename(full_page_image_path, meaningful_filename)
            meaningful_count += 1

            # Extract text from the meaningful image
            ocr_text = extract_text_from_image(meaningful_filename)

            # Save OCR text
            text_file = f"{extracted_text_dir}/page_{page_num + 1}.txt"
            with open(text_file, "w", encoding="utf-8") as tf:
                tf.write(ocr_text)
        else:
            os.remove(full_page_image_path)

    print(f"Filtered {meaningful_count} meaningful page images (charts/graphs).")

    # Save extracted page-wise text
    with open(f"{extracted_text_dir}/extracted_text.txt", "w", encoding="utf-8") as f:
        f.write(all_text)

    print("Full text and meaningful page images extraction complete.")
    return all_text, filtered_dir

# Example usage
pdf_path = "pdf/Q1FY24.pdf"  # Replace with your actual path
extract_pdf_text_and_images(pdf_path)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Filtered 35 meaningful page images (charts/graphs).
Full text and meaningful page images extraction complete.


('\n--- Page 1 ---\nCopyright © 2019, Tracxn Technologies Private Limited. All rights reserved.\nPrivate Market Intelligence Platform\nQ1 FY24 Investor Presentation\n 1st August 2023\n\n\n--- Page 2 ---\nBy attending this presentation including the commentary by the Company management \nand the transcript of the same, together (“Presentation”) and/or accepting a copy of it, \nyou agree to be bound by the foregoing limitations and conditions:\nThis Presentation is prepared by Tracxn Technologies Limited (“Company”) solely for \ninformation purposes and does not constitute an offer, solicitation, recommendation, or \ninvitation to purchase or subscribe for any securities and shall not form the basis of or \nbe relied on in connection with any contract or binding commitment whatsoever. This \nPresentation does not consider, nor does it provide any tax, legal, or investment advice \nor opinion regarding the specific investment objectives or financial situation of any \nperson. This Present

In [18]:
import os

extracted_text_dir = "/Users/gamaksahani/Documents/VSCodes/Assignment/extracted_text"
combined_file = os.path.join(extracted_text_dir, "combined_summary.txt")

with open(combined_file, "w", encoding="utf-8") as combined:
    with open(os.path.join(extracted_text_dir, "extracted_text.txt"), "r", encoding="utf-8") as pdf_text:
        combined.write("=== Full PDF Extracted Text ===\n\n")
        combined.write(pdf_text.read())
        combined.write("\n\n")
    combined.write("=== OCR Extracted from Page Images ===\n\n")
    for txt_file in sorted(os.listdir(extracted_text_dir)):
        if txt_file.startswith("page_") and txt_file.endswith(".txt"):
            with open(os.path.join(extracted_text_dir, txt_file), "r", encoding="utf-8") as ocr_text:
                combined.write(f"--- {txt_file} ---\n")
                combined.write(ocr_text.read())
                combined.write("\n\n")

print(f"Combined summary created at {combined_file}")


Combined summary created at /Users/gamaksahani/Documents/VSCodes/Assignment/extracted_text/combined_summary.txt
